In [ ]:
!pip install ripser

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import cv2
from skimage.filters import prewitt_h,prewitt_v
from skimage.filters import sato
from skimage.filters import sobel
from skimage import feature
from skimage import measure
from skimage.feature import hog
from skimage.filters.rank import entropy
from skimage.morphology import disk
from skimage.measure.entropy import shannon_entropy
from skimage.feature import corner_harris, corner_subpix, corner_peaks
from skimage import data, segmentation, color, filters, io
from skimage.future import graph

from skimage import color

from tensorflow.python.keras import models, layers, losses, optimizers, utils
from tensorflow.python.keras import backend as K


from ripser import ripser as rpser
from persim import plot_diagrams
from ripser import Rips



# for dirname, _, filenames in os.walk('/kaggle/input/lego-parts/parts_148/parts_148'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# def PINet_CIFAR10():
    
#     ## model
#     input_shape = [32,32,3]
#     initial_conv_width=3
#     initial_stride=1
#     initial_filters=64
#     initial_pool_width=3
#     initial_pool_stride=2
#     use_global_pooling = True
#     dropout_rate = 0.2

#     model_input = layers.Input(shape=input_shape)

#     x = layers.Conv2D(
#         128,
#         initial_conv_width,
#         strides=initial_stride,
#         padding="same")(model_input)
#     x = layers.BatchNormalization()(x)
#     x = layers.Activation("relu")(x)
#     x = layers.MaxPooling2D(
#         pool_size=initial_pool_width,
#         strides=initial_pool_stride,
#         padding="same")(x)

#     x = layers.Conv2D(
#         256,
#         initial_conv_width,
#         strides=initial_stride,
#         padding="same")(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.Activation("relu")(x)
#     x = layers.MaxPooling2D(
#         pool_size=initial_pool_width,
#         strides=initial_pool_stride,
#         padding="same")(x)

#     x = layers.Conv2D(
#         512,
#         initial_conv_width,
#         strides=initial_stride,
#         padding="same")(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.Activation("relu")(x)
#     x = layers.MaxPooling2D(
#         pool_size=initial_pool_width,
#         strides=initial_pool_stride,
#         padding="same")(x)

#     x = layers.Conv2D(
#         1024,
#         initial_conv_width,
#         strides=initial_stride,
#         padding="same")(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.Activation("relu")(x)

#     if use_global_pooling:
#         x = layers.GlobalAveragePooling2D()(x)


#     x_logits1 = layers.Dense(2500, activation="relu")(x)

#     x_logits1_reshape = layers.Reshape((1,50,50))(x_logits1)

#     x_logits1_reshape = layers.Permute((2,3,1))(x_logits1_reshape)

#     x_logits2 = layers.Conv2DTranspose(
#                             3,
#                             50,
#                             strides=initial_stride,
#                             padding="same")(x_logits1_reshape)
#     x_logits2 = layers.BatchNormalization()(x_logits2)
#     x_logits2 = layers.Activation("relu")(x_logits2)

#     model_output = layers.Flatten()(x_logits2)
    
#     model = models.Model(model_input, model_output)

#     return model

## Selecting parts and prepping dataset

In [ ]:
parts=pd.read_csv("/kaggle/input/lego-parts/parts.csv")
parts['file_name_poss']= parts['part_num']+".png"
parts.head()

In [ ]:
parts.shape

In [ ]:
#selecting parts with both entries in csv and part drawings
parts_files_path= "/kaggle/input/lego-parts/parts_148/parts_148"
possibilities=parts['file_name_poss'].tolist()
diagrams=os.listdir(parts_files_path)
complete_parts=list(set(possibilities) & set(diagrams))
len(complete_parts)

In [ ]:
parts_select=parts[parts['file_name_poss'].isin(complete_parts)]
parts_select.shape
parts_select

In [ ]:
parts_select[parts_select.duplicated('file_name_poss')]
# 5 part_names are duplicated

In [ ]:
duplicated_parts= ["4591.png","4588.png","4590.png","4740.png","4742.png"]
parts_select[parts_select['file_name_poss'].isin(duplicated_parts)]

In [ ]:
parts_select.drop_duplicates(subset ="file_name_poss", 
                     keep = False, inplace = True)
parts_select.shape

In [ ]:
#Distribution of data across part categories
percent_distr = round(parts_select["part_cat_id"].value_counts() / len(parts_select["part_cat_id"]) * 100,2)

print("Data Percent: ")
print(percent_distr)

In [ ]:
len(np.unique(parts_select["part_cat_id"]))

In [ ]:
complete_parts = [x for x in complete_parts if x not in duplicated_parts]

In [ ]:
len(complete_parts)

In [ ]:
for part in complete_parts[:20]:
    im = cv2.imread("/kaggle/input/lego-parts/parts_148/parts_148/" + part, cv2.IMREAD_GRAYSCALE)
    
    plt.imshow(im, cmap='gray')
    plt.show()

In [ ]:
def load_data(dir_data,part_df):
    ''' Load each of the image files into memory 

    While this is feasible with a smaller dataset, for larger datasets,
    not all the images would be able to be loaded into memory
    '''
    parts_df  = part_df
    labels    = parts_df.part_cat_id.values
    ids       = parts_df.part_num.values
    data      = []
    for identifier in ids:
        fname     = dir_data + identifier + '.png'
        image     = mpl.image.imread(fname)
        data.append(image)
    data = np.array(data) # Convert to Numpy array
    return data, labels

In [ ]:
dir_images="../input/lego-parts/parts_148/parts_148/"
data, labels = load_data(dir_images,parts_select)

## Feature extraction

### Grayscale conversion

In [ ]:
# using Luminance to obtain grayscale images
# luminance is the weighted average of RGB values
data_gray = [ color.rgb2gray(i) for i in data]

### Horizontal and vertical edges

In [ ]:
hor_edges = []
ver_edges = []
for image in data_gray:
    #calculating horizontal edges using prewitt kernel
    edges_prewitt_horizontal = prewitt_h(image)
    #calculating vertical edges using prewitt kernel
    edges_prewitt_vertical = prewitt_v(image)
    hor_edges.append(edges_prewitt_horizontal)
    ver_edges.append(edges_prewitt_vertical)


In [ ]:
# plot image after applying vertical edge filter to grayscale image
plt.imshow(ver_edges[855])
plt.axis('off')
plt.title('Image after applying vertical edge detector')

In [ ]:
# plot image after applying horizontal edge filter to grayscale image
plt.imshow(hor_edges[855])
plt.axis('off')
plt.title('Image after applying horizontal edge detector')

### Canny Edge Detection

In [ ]:
canny_edges_for_image = []
for image in data_gray:
    # running this bit to get the images after canny edge detector is applied- to visualize
    edges_sigma1 = feature.canny(image, sigma=3)
    canny_edges_for_image.append(edges_sigma1)
    
len(canny_edges_for_image)

In [ ]:
# plot image after applying canny edge detector to grayscale image
plt.imshow(canny_edges_for_image[855])
plt.axis('off')
plt.title('Image after applying Canny Edge Detector')

The Canny filter is a multi-stage edge detector. It uses a filter based on the derivative of a Gaussian in order to compute the intensity of the gradients.The Gaussian reduces the effect of noise present in the image. Then, potential edges are thinned down to 1-pixel curves by removing non-maximum pixels of the gradient magnitude. Finally, edge pixels are kept or removed using hysteresis thresholding on the gradient magnitude.

The Canny has three adjustable parameters: the width of the Gaussian (the noisier the image, the greater the width), and the low and high threshold for the hysteresis thresholding.

Source: [skimage canny](https://scikit-image.org/docs/dev/auto_examples/edges/plot_canny.html)

### Corner Detection

In [ ]:
#corner detection algos
harris_corner_for_image = []
corner_peaks_res=[]
corner_subpix_res=[]
for image in data_gray:
    coords = corner_harris(image)
    coords_peaks = corner_peaks(corner_harris(image), min_distance=5, threshold_rel=0.02)
    coords_subpix = corner_subpix(image, coords_peaks, window_size=13)
    harris_corner_for_image.append(coords)
    corner_peaks_res.append(coords_peaks)
    corner_subpix_res.append(coords_peaks)
    
len(harris_corner_for_image)

In [ ]:
# plot image after applying corner detection algos
plt.imshow(harris_corner_for_image[855])
plt.axis('off')
plt.title('Image after applying Harris Corner Detector')

In [ ]:
# plot image after applying corner detection algos
plt.imshow(corner_peaks_res[855])
plt.axis('off')
plt.title('Image after applying Find peaks in corner measure response image')

In [ ]:
# plot image after applying corner detection algos
plt.imshow(corner_subpix_res[855])
plt.axis('off')
plt.title('Image after applying subpixel position of corners')

### Entropy Filter

In [ ]:
#compute entropies- apply entropies filter
entropies = []
for image in data_gray:
    #getting images with entropy filter applied to the images
    e1 =entropy(image, disk(10))
    entropies.append(e1)
    
len(entropies)

In [ ]:
# plot image after applying entropy filter to grayscale image
plt.imshow(entropies[855])
plt.axis('off')
plt.title('Image after applying entropy filter')

The information entropy is the log-base-2 of the number of possible outcomes for a message.

For an image, local entropy is related to the complexity contained in a given neighborhood, typically defined by a structuring element. The entropy filter can detect subtle variations in the local gray level distribution.

Source: [skimage entropy](https://scikit-image.org/docs/dev/auto_examples/filters/plot_entropy.html)

### HOG (Histogram of Oriented Gradient) transformation

In [ ]:
# apply HOG (Histogram of Oriented Gradient) transformation
ppc = 16
hog_images = []
hog_features = []
for image in data_gray:
    # extracting hog transformed images and the feature arrays
    fd,hog_image = hog(image, orientations=10, pixels_per_cell=(ppc,ppc),cells_per_block=(4, 4),block_norm= 'L2',visualize=True)
    hog_images.append(hog_image)
    hog_features.append(fd)

In [ ]:
# plot image after applying HOG filter to grayscale image
plt.imshow(hog_images[855])
plt.axis('off')
plt.title('Image after applying HOG filter')

### SATO tubeness

In [ ]:
sato_res = []
for image in data_gray:
    #getting images with entropy filter applied to the images
    st1 =sato(image)
    sato_res.append(st1)
    
len(sato_res)

In [ ]:
# plot image after applying sato tube filter
plt.imshow(sato_res[855])
plt.axis('off')
plt.title('Image after applying Sato Tube filter')

This filter can be used to detect continuous ridges, e.g. tubes, wrinkles, rivers. It can be used to calculate the fraction of the whole image containing such objects.

Defined only for 2-D and 3-D images. Calculates the eigenvectors of the Hessian to compute the similarity of an image region to tubes.

Source: [skimage sato](https://scikit-image.org/docs/dev/api/skimage.filters.html#skimage.filters.sato)

In [ ]:
# import skimage
# for img in data_gray:
#     labels = segmentation.slic(img)
#     edge_map = filters.sobel(color.rgb2gray(img))
#     rag = graph.rag_boundary(labels, edge_map)
 
    
# NEED TO WORK ON THIS

In [ ]:
type(data_gray)

In [ ]:
np.array(data_gray[0]).shape

In [ ]:
rips = Rips()
diagrams = rips.fit_transform(np.array(data_gray[4]))
rips.plot(diagrams)

### Doing this for black lego parts (as it has a lot of parts)

In [ ]:
parts_black=pd.read_csv("/kaggle/input/black-lego-parts/black_final.csv")
parts_black['file_name_poss']= parts_black['part_num']+".png"


#selecting parts with both entries in csv and part drawings
figures_parts_files_path= "/kaggle/input/black-lego-parts/black/black/0"
possibilities=parts_black['file_name_poss'].tolist()
diagrams=os.listdir(figures_parts_files_path)
complete_parts=list(set(possibilities) & set(diagrams))
print(len(complete_parts))

# selecting by group figures
parts_select_fig=parts_black[parts_black['file_name_poss'].isin(complete_parts)]


In [ ]:
parts_select_fig.shape

In [ ]:
def load_data(dir_data,part_df):
    ''' Load each of the image files into memory 

    While this is feasible with a smaller dataset, for larger datasets,
    not all the images would be able to be loaded into memory
    '''
    parts_df  = part_df
    ids       = parts_df.part_num.values
    data      = []
    for identifier in ids:
        fname     = dir_data + identifier + '.png'
        image     = mpl.image.imread(fname)
        data.append(image)
    data = np.array(data) # Convert to Numpy array
    return data

In [ ]:
# Load data


figures_dir="../input/black-lego-parts/black/black/0/"
figures = load_data(figures_dir,parts_select_fig)


In [ ]:
# using Luminance to obtain grayscale images
# luminance is the weighted average of RGB values
data_gray_fig = [ color.rgb2gray(i) for i in figures]

In [ ]:
# plot image after applying vertical edge filter to grayscale image
plt.imshow(data_gray_fig[150], cmap='gray')
plt.axis('off')
plt.title('Image after applying grayscale filter')

In [ ]:
hor_edges = []
ver_edges = []
for image in data_gray_fig:
    #calculating horizontal edges using prewitt kernel
    edges_prewitt_horizontal = prewitt_h(image)
    #calculating vertical edges using prewitt kernel
    edges_prewitt_vertical = prewitt_v(image)
    hor_edges.append(edges_prewitt_horizontal)
    ver_edges.append(edges_prewitt_vertical)

In [ ]:
# plot image after applying vertical edge filter to grayscale image
plt.imshow(ver_edges[150])
plt.axis('off')
plt.title('Image after applying vertical edge detector')

In [ ]:
# plot image after applying horizontal edge filter to grayscale image
plt.imshow(hor_edges[150])
plt.axis('off')
plt.title('Image after applying horizontal edge detector')

In [ ]:
canny_edges_for_image = []
for image in data_gray_fig:
    # running this bit to get the images after canny edge detector is applied- to visualize
    edges_sigma1 = feature.canny(image, sigma=3)
    canny_edges_for_image.append(edges_sigma1)
    
len(canny_edges_for_image)

In [ ]:
# plot image after applying canny edge detector to grayscale image
plt.imshow(canny_edges_for_image[150])
plt.axis('off')
plt.title('Image after applying Canny Edge Detector')

In [ ]:
#compute entropies- apply entropies filter
entropies = []
for image in data_gray_fig:
    #getting images with entropy filter applied to the images
    e1 =entropy(image, disk(10))
    entropies.append(e1)
    
len(entropies)

In [ ]:
# plot image after applying entropy filter to grayscale image
plt.imshow(entropies[150])
plt.axis('off')
plt.title('Image after applying entropy filter')

In [ ]:
#corner detection algos
harris_corner_for_image = []
corner_peaks_res=[]
corner_subpix_res=[]
for image in data_gray_fig:
    coords = corner_harris(image)
    coords_peaks = corner_peaks(corner_harris(image), min_distance=5, threshold_rel=0.02)
    coords_subpix = corner_subpix(image, coords_peaks, window_size=13)
    harris_corner_for_image.append(coords)
    corner_peaks_res.append(coords_peaks)
    corner_subpix_res.append(coords_peaks)
    
len(harris_corner_for_image)

In [ ]:
# plot image after applying corner detection algos
plt.imshow(harris_corner_for_image[150])
plt.axis('off')
plt.title('Image after applying Harris Corner Detector')

In [ ]:
# apply HOG (Histogram of Oriented Gradient) transformation
ppc = 16
hog_images = []
hog_features = []
for image in data_gray_fig:
    # extracting hog transformed images and the feature arrays
    fd,hog_image = hog(image, orientations=10, pixels_per_cell=(ppc,ppc),cells_per_block=(4, 4),block_norm= 'L2',visualize=True)
    hog_images.append(hog_image)
    hog_features.append(fd)

In [ ]:
# plot image after applying HOG filter to grayscale image
plt.imshow(hog_images[150])
plt.axis('off')
plt.title('Image after applying HOG filter')

In [ ]:
sato_res = []
for image in data_gray_fig:
    #getting images with entropy filter applied to the images
    st1 =sato(image)
    sato_res.append(st1)
    
len(sato_res)

In [ ]:
# plot image after applying sato tube filter
plt.imshow(sato_res[150])
plt.axis('off')
plt.title('Image after applying Sato Tube filter')

In [ ]:
rips = Rips()
diagrams_gray = rips.fit_transform(np.array(data_gray_fig[150]))
rips.plot(diagrams_gray)
plt.title("Persistence Diagram from Grayscale Image (thersh=inf)")
plt.show()

In [ ]:
rips2 = Rips(thresh=0.8)
diagrams_gray_2 = rips2.fit_transform(np.array(data_gray_fig[150]))
rips2.plot(diagrams_gray_2)
plt.title("Persistence Diagram from Grayscale Image (threshold=0.8)")
plt.show()

In [ ]:
rips = Rips()
diagrams_sato = rips.fit_transform(np.array(sato_res[150]))
rips.plot(diagrams_sato)
plt.title("Persistence Diagram from Sato Tubeness Filter")
plt.show()

In [ ]:
rips = Rips()
diagrams_canny = rips.fit_transform(np.array(canny_edges_for_image[150]))
rips.plot(diagrams_canny)
plt.title("Persistence Diagram from Canny Edge Detected Image")
plt.show()

In [ ]:
rips = Rips()
diagrams_prewitt_h = rips.fit_transform(np.array(hor_edges[150]))
rips.plot(diagrams_prewitt_h)
plt.title("Persistence Diagram from Prewitt Horizontal Edges Image")
plt.show()

In [ ]:
rips = Rips()
diagrams_prewitt_v = rips.fit_transform(np.array(ver_edges[150]))
rips.plot(diagrams_prewitt_v)
plt.title("Persistence Diagram from Prewitt Vertical Edges Image")
plt.show()

In [ ]:
fig=plt.figure(figsize=(40,30))
fig, ax = plt.subplots(2,3)
fig.tight_layout()
rips.plot(diagrams_gray,ax=plt.subplot(231), title="Grayscale(thresh=inf)")
rips2.plot(diagrams_gray_2,ax=plt.subplot(232), title="Grayscale(thresh=0.8)")
rips.plot(diagrams_canny,ax=plt.subplot(233), title="Canny Edge Detected")
rips.plot(diagrams_prewitt_h,ax=plt.subplot(234), title="Prewitt Hor.")
rips.plot(diagrams_prewitt_v,ax=plt.subplot(235), title="Prewitt Ver.")
rips.plot(diagrams_sato,ax=plt.subplot(236), title="SATO Tubeness")

plt.show()

In [ ]:
#selecting parts with both entries in csv and part drawings
bricks_parts_files_path= "/kaggle/input/black-lego-parts/black/black/1"
possibilities=parts_black['file_name_poss'].tolist()
diagrams=os.listdir(bricks_parts_files_path)
complete_parts=list(set(possibilities) & set(diagrams))
print(len(complete_parts))

# selecting by group bricks
parts_select_brick=parts_black[parts_black['file_name_poss'].isin(complete_parts)]

# Load data
bricks_dir="../input/black-lego-parts/black/black/1/"
bricks = load_data(bricks_dir,parts_select_brick)

In [ ]:
# using Luminance to obtain grayscale images
# luminance is the weighted average of RGB values
data_gray_brick = [ color.rgb2gray(i) for i in bricks]

In [ ]:
# plot image after applying vertical edge filter to grayscale image
plt.imshow(data_gray_brick[10], cmap='gray')
plt.axis('off')
plt.title('Image after applying grayscale filter')

In [ ]:
canny_edges_for_image = []
for image in data_gray_brick:
    # running this bit to get the images after canny edge detector is applied- to visualize
    edges_sigma1 = feature.canny(image, sigma=3)
    canny_edges_for_image.append(edges_sigma1)
    
len(canny_edges_for_image)

In [ ]:
# plot image after applying canny edge detector to grayscale image
plt.imshow(canny_edges_for_image[10])
plt.axis('off')
plt.title('Image after applying Canny Edge Detector')

In [ ]:
hor_edges = []
ver_edges = []
for image in data_gray_brick:
    #calculating horizontal edges using prewitt kernel
    edges_prewitt_horizontal = prewitt_h(image)
    #calculating vertical edges using prewitt kernel
    edges_prewitt_vertical = prewitt_v(image)
    hor_edges.append(edges_prewitt_horizontal)
    ver_edges.append(edges_prewitt_vertical)

In [ ]:
fig=plt.figure(figsize=(40,30))
fig, ax = plt.subplots(1,3)
fig.tight_layout()
ax[0].imshow(canny_edges_for_image[10])
ax[0].set_title("Canny Edge")
ax[0].axis('off')
ax[1].imshow(hor_edges[10])
ax[1].set_title("Prewitt Hor. Edge")
ax[1].axis('off')
ax[2].imshow(ver_edges[10])
ax[2].set_title("Prewitt Ver. Edge")
ax[2].axis('off')
#plt.suptitle("Examples of Edge Detection from category Bricks")
plt.show()

In [ ]:
#compute entropies- apply entropies filter
entropies = []
for image in data_gray_brick:
    #getting images with entropy filter applied to the images
    e1 =entropy(image, disk(10))
    entropies.append(e1)
    
len(entropies)

In [ ]:
#selecting parts with both entries in csv and part drawings
transport_parts_files_path= "/kaggle/input/black-lego-parts/black/black/3"
possibilities=parts_black['file_name_poss'].tolist()
diagrams=os.listdir(transport_parts_files_path)
complete_parts=list(set(possibilities) & set(diagrams))
print(len(complete_parts))

# selecting by group transport
parts_select_transport=parts_black[parts_black['file_name_poss'].isin(complete_parts)]

# Load data
transport_dir="../input/black-lego-parts/black/black/3/"
transports = load_data(transport_dir,parts_select_transport)

In [ ]:
# using Luminance to obtain grayscale images
# luminance is the weighted average of RGB values
data_gray_transport = [ color.rgb2gray(i) for i in transports]

In [ ]:
# plot image after applying grayscale image
plt.imshow(data_gray_transport[76], cmap='gray')
plt.axis('off')
plt.title('Image after applying grayscale filter')

In [ ]:
canny_edges_for_image = []
for image in data_gray_transport:
    # running this bit to get the images after canny edge detector is applied- to visualize
    edges_sigma1 = feature.canny(image, sigma=3)
    canny_edges_for_image.append(edges_sigma1)
    
len(canny_edges_for_image)

In [ ]:
# plot image after applying canny edge detector to grayscale image
plt.imshow(canny_edges_for_image[76])
plt.axis('off')
plt.title('Image after applying Canny Edge Detector')

In [ ]:
hor_edges = []
ver_edges = []
for image in data_gray_transport:
    #calculating horizontal edges using prewitt kernel
    edges_prewitt_horizontal = prewitt_h(image)
    #calculating vertical edges using prewitt kernel
    edges_prewitt_vertical = prewitt_v(image)
    hor_edges.append(edges_prewitt_horizontal)
    ver_edges.append(edges_prewitt_vertical)

In [ ]:
fig=plt.figure(figsize=(40,30))
fig, ax = plt.subplots(1,3)
fig.tight_layout()
ax[0].imshow(canny_edges_for_image[76])
ax[0].set_title("Canny Edge")
ax[0].axis('off')
ax[1].imshow(hor_edges[76])
ax[1].set_title("Prewitt Hor. Edge")
ax[1].axis('off')
ax[2].imshow(ver_edges[76])
ax[2].set_title("Prewitt Ver. Edge")
ax[2].axis('off')
#plt.suptitle("Examples of Edge Detection from category Bricks")
plt.show()

In [ ]:
#compute entropies- apply entropies filter
entropies = []
for image in data_gray_transport:
    #getting images with entropy filter applied to the images
    e1 =entropy(image, disk(10))
    entropies.append(e1)
    
len(entropies)

In [ ]:
# plot image after applying entropy filter to grayscale image
plt.imshow(entropies[76])
plt.axis('off')
plt.title('Image after applying entropy filter')

In [ ]:
#selecting parts with both entries in csv and part drawings
mechanical_parts_files_path= "/kaggle/input/black-lego-parts/black/black/2"
possibilities=parts_black['file_name_poss'].tolist()
diagrams=os.listdir(mechanical_parts_files_path)
complete_parts=list(set(possibilities) & set(diagrams))
print(len(complete_parts))

# selecting by group mech
parts_select_mechanical=parts_black[parts_black['file_name_poss'].isin(complete_parts)]

# Load data
mech_dir="../input/black-lego-parts/black/black/2/"
mechs = load_data(mech_dir,parts_select_mechanical)

In [ ]:
# using Luminance to obtain grayscale images
# luminance is the weighted average of RGB values
data_gray_mech = [ color.rgb2gray(i) for i in mechs]

In [ ]:
# plot image after applying grayscale image
plt.imshow(data_gray_mech[188], cmap='gray')
plt.axis('off')
plt.title('Image after applying grayscale filter')

In [ ]:
#selecting parts with both entries in csv and part drawings
panels_parts_files_path= "/kaggle/input/black-lego-parts/black/black/4"
possibilities=parts_black['file_name_poss'].tolist()
diagrams=os.listdir(panels_parts_files_path)
complete_parts=list(set(possibilities) & set(diagrams))
print(len(complete_parts))

# selecting by group panels
parts_select_panels=parts_black[parts_black['file_name_poss'].isin(complete_parts)]

# Load data
panels_dir="../input/black-lego-parts/black/black/4/"
panels = load_data(panels_dir,parts_select_panels)

In [ ]:
# using Luminance to obtain grayscale images
# luminance is the weighted average of RGB values
data_gray_panel = [ color.rgb2gray(i) for i in panels]

In [ ]:
# plot image after applying grayscale image
plt.imshow(data_gray_panel[54], cmap='gray')
plt.axis('off')
plt.title('Image after applying grayscale filter')

In [ ]:
#selecting parts with both entries in csv and part drawings
others_parts_files_path= "/kaggle/input/black-lego-parts/black/black/5"
possibilities=parts_black['file_name_poss'].tolist()
diagrams=os.listdir(others_parts_files_path)
complete_parts=list(set(possibilities) & set(diagrams))
print(len(complete_parts))

# selecting by group others
parts_select_others=parts_black[parts_black['file_name_poss'].isin(complete_parts)]

# Load data
others_dir="../input/black-lego-parts/black/black/5/"
others = load_data(others_dir,parts_select_others)

In [ ]:
# using Luminance to obtain grayscale images
# luminance is the weighted average of RGB values
data_gray_other = [ color.rgb2gray(i) for i in others]

In [ ]:
# plot image after applying grayscale image
plt.imshow(data_gray_other[450], cmap='gray')
plt.axis('off')
plt.title('Image after applying grayscale filter')

In [ ]:
rips = Rips()
diagrams_gray_fig = rips.fit_transform(np.array(data_gray_fig[150]))
diagrams_gray_brick = rips.fit_transform(np.array(data_gray_brick[10]))
diagrams_gray_mech = rips.fit_transform(np.array(data_gray_mech[188]))
diagrams_gray_transport = rips.fit_transform(np.array(data_gray_transport[76]))
diagrams_gray_panel = rips.fit_transform(np.array(data_gray_panel[54]))
diagrams_gray_other = rips.fit_transform(np.array(data_gray_other[450]))

In [ ]:
# sample persistence diagram from each category
fig=plt.figure(figsize=(60,50))
fig, ax = plt.subplots(2,3)
fig.tight_layout(pad=2)
rips.plot(diagrams_gray_fig,ax=plt.subplot(231), title="Figures Grayscale")
rips.plot(diagrams_gray_brick,ax=plt.subplot(232), title="Bricks Grayscale")
rips.plot(diagrams_gray_mech,ax=plt.subplot(233), title="Mechanical Grayscale")
rips.plot(diagrams_gray_transport,ax=plt.subplot(234), title="Transport Grayscale")
rips.plot(diagrams_gray_panel,ax=plt.subplot(235), title="Panels Grayscale")
rips.plot(diagrams_gray_other,ax=plt.subplot(236), title="Others Grayscale")
plt.suptitle("Persistence Diagrams")
plt.show()

In [ ]:
fig=plt.figure(figsize=(60,50))
fig, ax = plt.subplots(2,3)
fig.tight_layout(pad=2)
ax[0,0].imshow(data_gray_fig[150], cmap='gray')
ax[0,0].axis('off')
ax[0,0].set_title('Figures Grayscale')
ax[0,1].imshow(data_gray_brick[10], cmap='gray')
ax[0,1].axis('off')
ax[0,1].set_title('Bricks Grayscale')
ax[0,2].imshow(data_gray_mech[188], cmap='gray')
ax[0,2].axis('off')
ax[0,2].set_title('Mechanical Grayscale')
ax[1,0].imshow(data_gray_transport[76], cmap='gray')
ax[1,0].axis('off')
ax[1,0].set_title('Transport Grayscale')
ax[1,1].imshow(data_gray_panel[54], cmap='gray')
ax[1,1].axis('off')
ax[1,1].set_title('Panels Grayscale')
ax[1,2].imshow(data_gray_other[450], cmap='gray')
ax[1,2].axis('off')
ax[1,2].set_title('Others Grayscale')
plt.suptitle("Base Object Images")
plt.show()